Aluno: Bruno Ricardo Pereira dos Santos Santos

Nº USP: 10288640

In [ ]:
# Importando bibliotecas
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install plotly.express
from plotly import graph_objs as go

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


 # Identificação do problema

A base usada contém o texto de um SMS e a sua descrição, se é um spam ou não. O objetivo deste projeto é analisar e criar um modelo para classificação de textos de SMS.


In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/graduação/mineracao_dados_nao_estruturados/SMSSpamCollection.xlsx')
dataset

dataset['text'] = dataset['text'].astype(str)

# Pré-processamento

## Modelo bag-of-words

In [ ]:
# remoção de pontuacao e stopwords

def remove_stopwords(text,lang,domain_stopwords=[]):
  
  stop_words = nltk.corpus.stopwords.words(lang) # lang='portuguese' or lang='english'
  
  s = str(text).lower() # tudo para caixa baixa
  table = str.maketrans({key: None for key in string.punctuation})
  s = s.translate(table) # remove pontuacao
  tokens = word_tokenize(s) #obtem tokens
  v = [i for i in tokens if not i in stop_words and not i in domain_stopwords and not i.isdigit()] # remove stopwords
  s = ""
  for token in v:
    s += token+" "
  return s.strip()

# stemming
def stemming(text,lang):
  
  stemmer = PorterStemmer() # stemming para ingles
  
  if lang=='portuguese':
    stemmer = nltk.stem.RSLPStemmer() # stemming para portuguese
    
  tokens = word_tokenize(text) #obtem tokens
  
  sentence_stem = ''
  doc_text_stems = [stemmer.stem(i) for i in tokens]
  for stem in doc_text_stems:
    sentence_stem += stem+" "
    
  return sentence_stem.strip()

In [ ]:
# Bag of Words com refinamento: ponderação por TF-IDF
def compute_vsm_tfidf(dataset,lang,domain_stopwords=[]):
  
  d = []
  for index,row in dataset.iterrows():
    text = row['text'] #texto do evento
    text2 = remove_stopwords(text, lang,domain_stopwords)
    text3 = stemming(text2, lang)
    d.append(text3)
  matrix = TfidfVectorizer()
  X = matrix.fit_transform(d)
  
  tfidf_vect_df = pd.DataFrame(X.todense(), columns=matrix.get_feature_names())

  return tfidf_vect_df


vsm = compute_vsm_tfidf(dataset,'english')
vsm

,0089mi,0776xxxxxxx,077xxx,0789xxxxxxx,0796xxxxxx,07xxxxxxxxx,08452810075over18,08700621170150p,08701417012150p,08702840625comuk,08704439680tsc,0870737910216yr,0870k,087123002209am7pm,08712460324nat,0871277810710pmin,0871277810910pmin,087143423992stop,087147123779am7pm,08717890890â,08718726270150gbpmtmsg18,09065171142stopsms08,09065171142stopsms08718727870150ppm,09066649731from,0anetwork,10,100,1000,10000,100000,1000call,100psm,101mega,10am,10am7pm,10am9pm,10k,10p,10pmin,10ppm,...,youthat,youto,youuuuu,youv,youwanna,youwhen,youâ,yovil,yowif,yoyyooo,yr,ystrdayic,yummi,yummmm,yun,yunni,yuo,yuou,yup,yupz,ywhere,zac,zaher,zealand,zebra,zed,zero,zhong,zindgi,zoe,zogtoriu,zoom,zouk,zyada,¾ã,ã¼,ã¼ll,ãœ,ãœll,œharri
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.247075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.289146,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.386368,0.0,0.0,0.0,0.0
5571,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5572,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

## Word2Vec

Tendo em vista que nossa base de dados não apresenta em sua maior parte: 

- Textos muito complexos
- Palavras com mais de um sentido
- Palavras que invertem o sentido da sentença

Não é necessário usar uma Word Embedding mais robusta como o ELMO ou o BERT para obter um resultado satisfatório, podemos usar uma word Embedding estática. 

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-11-02 23:04:34--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.89.37
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.89.37|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
!pip install gensim
from gensim.models import KeyedVectors

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
df_w2v_train, df_w2v_test = train_test_split(dataset, test_size=0.30, random_state=42, stratify=dataset.target)

In [ ]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
doc_embeddings = []
for index,row in df_w2v_train.iterrows():
  #print(row['OriginalTweet'])
  sentence = preprocess(row['text'])
  #print(sentence)
  L = []
  for token in sentence:
    try:
      #print(token,word2vec[token])
      L.append(word2vec[token])
    except:
      1
  if len(L) > 0: tweet_vec = np.mean(np.array(L),axis=0)
  else: tweet_vec = np.zeros(300)
  doc_embeddings.append(tweet_vec)

In [ ]:
X_w2v_train = np.array(doc_embeddings)
y_w2v_train = df_w2v_train.target.to_list()
X_w2v_train.shape

(3901, 300)

In [ ]:
pd.DataFrame(X_w2v_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.067671,-0.002075,0.154088,0.229597,-0.057103,0.000218,0.101074,-0.101353,0.070304,0.048741,-0.071184,-0.232701,-0.092721,-0.035191,-0.062273,0.205497,0.113491,0.072353,-0.023333,-0.086914,-0.034110,0.061611,0.212472,0.079520,0.072043,0.068464,-0.043736,0.134975,-0.055725,-0.047538,-0.014186,0.148211,-0.014784,-0.092861,-0.121443,-0.056745,0.025495,0.009905,0.013048,0.015520,...,-0.073696,0.096645,0.060704,-0.074812,0.004185,0.052804,-0.120640,0.016615,0.027902,0.019741,-0.014230,0.178667,0.107030,0.002241,0.125907,-0.100359,-0.071058,-0.071217,-0.087716,0.077881,-0.020996,-0.030829,-0.016009,0.103289,-0.014526,0.013236,-0.098430,0.020473,0.007690,0.025984,0.073172,0.191267,-0.124233,0.088693,-0.049997,-0.209612,0.004185,-0.013088,-0.020721,0.013532
1,0.009888,0.053017,-0.014893,-0.068359,-0.029907,-0.125488,0.085205,-0.017090,0.027832,0.065674,-0.075439,-0.153809,0.027893,0.049377,-0.117065,0.109741,0.049042,0.212402,-0.116211,0.016602,-0.014160,0.072968,0.250488,-0.027405,0.291992,-0.021065,-0.014771,-0.009979,0.051514,0.054626,0.051880,-0.017029,0.007324,-0.002441,0.091187,0.027222,0.101562,-0.026070,-0.171204,0.043823,...,-0.090851,-0.084961,0.084473,0.052979,0.047302,0.359863,0.143555,-0.046631,-0.010254,0.083984,0.200195,-0.041992,0.145996,-0.045013,0.073669,-0.059204,0.003174,-0.142487,-0.150879,-0.064941,-0.132446,-0.025879,-0.096069,-0.065186,0.101929,0.035034,-0.052246,-0.069901,0.049286,0.000854,-0.012970,-0.052612,-0.090698,0.115479,-0.154541,-0.020874,0.092529,-0.115173,0.066711,0.026489
2,-0.028809,0.020508,0.122314,0.075562,-0.157837,0.182617,0.058716,-0.056410,0.262207,0.100342,-0.143799,-0.317383,-0.172974,-0.075745,-0.016541,0.253418,0.028809,0.043701,-0.075989,0.080078,-0.198730,0.079590,0.177246,-0.108185,-0.023193,-0.013824,-0.161621,0.087158,-0.003357,-0.048035,0.006104,0.203125,-0.204620,-0.161377,-0.100342,0.128418,-0.017090,-0.098877,0.096191,-0.134033,...,0.021118,0.047852,-0.056641,0.034180,-0.003235,0.089844,-0.171631,-0.074829,0.198486,-0.191895,0.042480,0.067200,0.187988,0.069580,0.002930,-0.133789,0.042542,-0.172363,-0.119598,0.085938,-0.079834,-0.026520,-0.020447,0.135742,0.100708,0.017822,-0.091919,-0.096436,0.153809,0.100098,-0.135498,0.096191,-0.048340,0.114746,-0.107544,-0.181519,-0.236816,-0.049927,-0.063837,0.000977
3,0.053874,0.004395,-0.021566,0.118001,-0.094564,0.043538,0.193848,-0.239583,-0.043294,0.176758,-0.014445,-0.019857,0.070028,-0.012533,-0.080566,0.107259,-0.037435,0.139974,0.053355,-0.102051,-0.095703,-0.011912,0.091777,-0.053385,0.128906,-0.027018,0.048950,0.101888,0.120605,-0.101237,-0.082845,0.190999,0.088521,0.041829,-0.090169,0.101481,0.001302,-0.076660,-0.007894,0.261312,...,0.058512,-0.046387,-0.001139,0.095296,0.044434,0.043050,0.004395,0.009237,-0.056315,0.062500,0.041911,0.081305,0.037109,0.107910,0.181641,-0.114176,-0.101807,-0.030477,-0.135905,-0.000081,0.108927,-0.031372,-0.064453,0.142741,-0.007650,-0.110189,-0.135986,0.083984,0.078776,0.020671,0.013672,-0.015299,-0.144694,-0.179769,-0.016032,-0.097168,-0.027837,-0.047526,0.011230,0.012655
4,0.193359,-0.004333,-0.032227,0.139648,-0.020264,-0.100098,-0.062500,-0.002060,0.152344,0.061279,-0.129883,-0.143555,-0.025024,0.131836,-0.109863,0.028809,0.027710,0.139648,-0.166992,-0.086426,-0.006805,-0.109863,0.062256,-0.057129,0.022339,0.050049,-0.092773,-0.033936,-0.020752,-0.218750,0.021606,0.022827,0.137695,-0.122070,0.050049,-0.118164,0.137695,-0.124023,0.043457,-0.060303,...,0.045898,-0.081543,-0.135742,-0.005219,0.086426,0.091797,-0.028320,0.079590,0.002762,0.010498,0.222656,0.141602,-0.010254,0.183594,-0.005707,-0.023682,-0.117188,-0.048584,-0.068359,-0.195312,0.101074,-0.028809,0.048828,-0.022583,-0.065430,0.043457,-0.041260,0.057861,-0.0451

In [ ]:
doc_embeddings = []
for index,row in df_w2v_test.iterrows():
  sentence = preprocess(row['text'])
  L = []
  for token in sentence:
    try:
      #print(token,word2vec[token])
      L.append(word2vec[token])
    except:
      1
  if len(L) > 0: tweet_vec = np.mean(np.array(L),axis=0)
  else: tweet_vec = np.zeros(300)
  doc_embeddings.append(tweet_vec)

In [ ]:
X_w2v_test = np.array(doc_embeddings)
y_w2v_test = df_w2v_test.target.to_list()
X_w2v_test.shape

(1673, 300)

# Extração de padrões (ML)

## Bag of Words

In [ ]:
# Preparando dados para classificador kNN
X = np.array(vsm)
length = np.sqrt((X**2).sum(axis=1))[:,None]
X_bag = X / (length+0.00001)

X.shape

(5574, 7593)

In [ ]:
Y_bag = dataset['target'].to_list()

In [ ]:
X_bag_train, X_bag_test, y_bag_train, y_bag_test = train_test_split(X_bag, Y_bag, test_size=0.30, random_state=42, stratify = Y_bag)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_bag_train, y_bag_train)

neigh.score(X_bag_test,y_bag_test)

predictions_bag = list(neigh.predict(X_bag_test))

In [ ]:

print('Accuracy score: {}'.format(accuracy_score(y_bag_test, predictions_bag)))
print('Precision score: {}'.format(precision_score(y_bag_test, predictions_bag,pos_label='spam')))
print('Recall score: {}'.format(recall_score(y_bag_test, predictions_bag,pos_label='spam')))
print('F1 score: {}'.format(f1_score(y_bag_test, predictions_bag,pos_label='spam')))

Accuracy score: 0.9282725642558278
Precision score: 1.0
Recall score: 0.4642857142857143
F1 score: 0.6341463414634146


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_bag_test,predictions_bag)

array([[1449,    0],
       [ 120,  104]])

## Word2Vec

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_w2v_train, y_w2v_train)

neigh.score(X_w2v_test,y_w2v_test)

predictions_w2v = list(neigh.predict(X_w2v_test))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(y_w2v_test, predictions_w2v)))
print('Precision score: {}'.format(precision_score(y_w2v_test, predictions_w2v,pos_label='spam')))
print('Recall score: {}'.format(recall_score(y_w2v_test, predictions_w2v,pos_label='spam')))
print('F1 score: {}'.format(f1_score(y_w2v_test, predictions_w2v,pos_label='spam')))

Accuracy score: 0.9563658099222953
Precision score: 0.8007968127490039
Recall score: 0.8973214285714286
F1 score: 0.8463157894736842


In [ ]:
confusion_matrix(y_w2v_test,predictions_w2v)

array([[1399,   50],
       [  23,  201]])

## BERT

Mesmo obtendo um modelo satisfatório usando Word2Vec, testaremos um modelo que demanda mais poder computacional:

In [ ]:
#!pip install ktrain
import keras
import ktrain
from ktrain import text

In [ ]:
(x_bert_train, y_bert_train), (x_bert_test, y_bert_test), preproc = text.texts_from_df(dataset, 
                                                                   'text',
                                                                   label_columns='target',
                                                                   maxlen=64, 
                                                                   max_features=10000,
                                                                   preprocess_mode='bert',
                                                                   lang=None,
                                                                   val_pct = 0.3,
                                                                   random_state=42
                                                                   )

['ham', 'spam']
      ham  spam
4977  1.0   0.0
650   0.0   1.0
5184  1.0   0.0
708   1.0   0.0
3295  1.0   0.0
['ham', 'spam']
      ham  spam
3690  1.0   0.0
3527  1.0   0.0
724   1.0   0.0
3370  1.0   0.0
468   1.0   0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', (x_bert_train, y_bert_train) , preproc=preproc)
classifier = ktrain.get_learner(model, 
                             train_data=(x_bert_train, y_bert_train), 
                             val_data=(x_bert_test, y_bert_test),
                             batch_size=64
                             )

Is Multi-Label? False
maxlen is 64
done.


In [ ]:
classifier.fit_onecycle(0.00002,2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
61/61 [==============================] - 3008s 49s/step - loss: 0.4402 - accuracy: 0.7765 - val_loss: 0.1162 - val_accuracy: 0.9809
Epoch 2/5
61/61 [==============================] - 2934s 48s/step - loss: 0.0542 - accuracy: 0.9859 - val_loss: 0.0286 - val_accuracy: 0.9934
Epoch 3/5
22/61 [=========>....................] - ETA: 27:59 - loss: 0.0248 - accuracy: 0.9922

KeyboardInterrupt: ignored

In [ ]:
resul = classifier.validate()
resul

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1447
           1       0.99      0.97      0.98       226

    accuracy                           0.99      1673
   macro avg       0.99      0.98      0.99      1673
weighted avg       0.99      0.99      0.99      1673



array([[1445,    2],
       [   7,  219]])

In [ ]:
classifier_model = ktrain.get_predictor(classifier.model, preproc)

In [ ]:
classifier_model.save('/content/drive/MyDrive/graduação/mineracao_dados_nao_estruturados/my_model')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



# Pós - processamento

- Acurácia: indica uma performance geral do modelo. Dentre todas as classificações, quantas o modelo classificou corretamente;

- Precisão: dentre todas as classificações de classe Positivo que o modelo fez, quantas estão corretas;

- Recall: dentre todas as situações de classe Positivo como valor esperado, quantas estão corretas;

- F1-Score: média harmônica entre precisão e recall.

Ao receber spams em SMS, é comum que eles contenham vírus ou busquem informações confidenciais do usuário. Sendo assim, o melhor modelo seria aquele que classifique corretamente spams o máximo possível, mesmo que para isso ele sugira que mensagens normais sejam spams também.

Neste caso, a melhor métrica para avaliar seria Recall

In [ ]:
d_2 = { " ": ['Acurácia', 'Precisão', 'Recall', 'F1 score'],
    "Modelo Bag of Words": [accuracy_score(y_bag_test, predictions_bag),
                            precision_score(y_bag_test, predictions_bag,pos_label='spam'),
                            recall_score(y_bag_test, predictions_bag,pos_label='spam'),
                            f1_score(y_bag_test, predictions_bag,pos_label='spam')],
    "Modelo Word2Vec": [accuracy_score(y_w2v_test, predictions_w2v),
                        precision_score(y_w2v_test, predictions_w2v,pos_label='spam'),
                        recall_score(y_w2v_test, predictions_w2v,pos_label='spam'),
                        f1_score(y_w2v_test, predictions_w2v,pos_label='spam')]
}
df_2 = pd.DataFrame(data = d_2)
df_2

,,Modelo Bag of Words,Modelo Word2Vec
0,Acurácia,0.928273,0.956366
1,Precisão,1.000000,0.800797
2,Recall,0.464286,0.897321
3,F1 score,0.634146,0.846316


In [ ]:
confusion_matrix(y_bag_test,predictions_bag)

array([[1449,    0],
       [ 120,  104]])

In [ ]:
confusion_matrix(y_w2v_test,predictions_w2v)

array([[1399,   50],
       [  23,  201]])

O modelo usando Bag of Words, apesar de ter uma boa acurácia, não foi um bom modelo para o nosso problema, devido ao seu baixo Recall.

Fazendo uma comparação, o modelo Bag of Words teve 120 casos de falso negativo, enquanto o Word2Vec teve 23 casos, em um total de 1673 casos de teste. 

Considerando uma cidade com uma população de 100 mil habitantes que recebem um SMS por dia, teríamos 7173 pessoas recebendo Spam sem serem alertados no primeiro modelo, já no segundo teríamos 1375. 

Por fim, o vencedor seria o modelo Word2Vector.

Este resultado consideou também o custo computacional. A base utilizada possui menos de 6000 SMS, porém, no "mundo real" teríamos um a base com uma quantidade muito superior de dados.

Deixando de lado o custo computacional, utilizando o modelo BERT, tivemos resultados muito superiores

In [ ]:
resul

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1447
           1       0.99      0.97      0.98       226

    accuracy                           0.99      1673
   macro avg       0.99      0.98      0.99      1673
weighted avg       0.99      0.99      0.99      1673



array([[1445,    2],
       [   7,  219]])

Usando o exemplo anterior da cidade com 100 mil habitantes, com o BERT teríamos 418 recebendo Spam sem alerta, um valor 3 vezes menor que o Word2Vec e 17 vezes menor que o Bag of Words

# Uso do conhecimento

Neste trabalho extraimos padrões de um conjunto de SMS, sendo assim, a aplicação mais óbvia é implementar o modelo no aplicativo de mensagens do usuário para que ele seja notificado caso aquele SMS tenha características de Spam, porém, isso também poderia ser feito com emails e caixas de mensagens em redes sociais como facebook e instagram.

Refazer: : Em relação ao pre-processamneto, poderia ter variado o max_features da BoW e utilizar o tfidf. Em relação a classificação, poderia ter variado alguns parâmetros do método (número de vizinhos no kNN) e utilizar outros métodos interessantes para classificação para compara-los. No mundo real, pode ser mais facil para o usuario identificar Spams no meio de mensagem
reais que ele vai olhar com calma porque são importantes, em vez de procurar pro mensagens reais no meio de varias Spams (mensagens
nao importantes)? Considerar a F1-Score invés do recall? 